In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-04 15:39:14--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  19.7MB/s    in 99s     

2021-01-04 15:40:54 (18.6 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-04 15:40:54--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [2]:
!cp ../drive/MyDrive/slt/late-fusion-duo/*.py signjoey/

In [3]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q ../drive/MyDrive/slt/keypoints/dev_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/train_features.zip -d data
!unzip -q ../drive/MyDrive/slt/keypoints/test_features.zip -d data

In [4]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

## Hidden features

In [5]:
for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)

  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [01:17<00:00, 91.02it/s]


## Change config files

In [7]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 1024]

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Run

In [8]:
!cp -r '../drive/MyDrive/slt/models/baseline_1' baseline
!cp -r '../drive/MyDrive/slt/models/hidden_features_1' features

In [9]:
import os
import sys
from signjoey.prediction import test

test(cfg_file="configs/sign.yaml", ckpts=["baseline/best.ckpt", "features/best.ckpt"], output_path="")

2021-01-04 15:57:26,188 - ------------------------------------------------------------
2021-01-04 15:57:26,192 - [DEV] partition [RECOGNITION] experiment [BW]: 1
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
2021-01-04 15:57:56,383 - finished in 30.1909s 
2021-01-04 15:57:56,385 - ************************************************************
2021-01-04 15:57:56,385 - [DEV] partition [RECOGNITION] results:
	New Best CTC Decode Beam Size: 1
	WER 65.07	(DEL: 63.17,	INS: 0.35,	SUB: 1.55)
2021-01-04 15:57:56,386 - ************************************************************
2021-01-04 15:57:56,387 - ------------------------------------------------------------
2021-01-04 15:57:56,388 - [DEV] partition [RECOGNITION] exper

translation


2021-01-04 16:16:19,429 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 1 and Alpha: -1
	BLEU-4 20.29	(BLEU-1: 46.17,	BLEU-2: 32.92,	BLEU-3: 25.12,	BLEU-4: 20.29)
	CHRF 43.81	ROUGE 46.33
2021-01-04 16:16:19,430 - ------------------------------------------------------------
2021-01-04 16:19:44,145 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: -1
	BLEU-4 20.54	(BLEU-1: 45.45,	BLEU-2: 32.63,	BLEU-3: 25.19,	BLEU-4: 20.54)
	CHRF 43.21	ROUGE 46.29
2021-01-04 16:19:44,146 - ------------------------------------------------------------
2021-01-04 16:20:44,617 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: 1
	BLEU-4 20.57	(BLEU-1: 45.63,	BLEU-2: 32.75,	BLEU-3: 25.25,	BLEU-4: 20.57)
	CHRF 43.29	ROUGE 46.26
2021-01-04 16:20:44,618 - ------------------------------------------------------------
2021-01-04 16:21:14,995 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 a